In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import census_key
c = Census(census_key, year=2018)

In [108]:
#Import covid data 
covid_91120_data=pd.read_csv('CSV_Data/09-11-2020.csv')
covid_91120_data=covid_91120_data.loc[covid_91120_data['FIPS'].isnull()==False]
covid_91120_data['FIPS']=covid_91120_data['FIPS'].astype(int).astype(str)
covid_91120_data.head()

covid_91120_data.loc[covid_91120_data['Admin2']=='St. Louis']



,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
2046,27137,St. Louis,Minnesota,US,2020-09-12 04:25:05,47.604841,-92.468799,1090,25,0,1065.0,"St. Louis, Minnesota, US",547.546089,2.293578
2247,29189,St. Louis,Missouri,US,2020-09-12 04:25:05,38.637700,-90.445837,21343,763,0,20580.0,"St. Louis, Missouri, US",2146.740360,3.574943


In [110]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

#https://api.census.gov/data/2018/acs/acs5/subject?get=NAME,S0101_C01_001E&for=county:*&key=YOUR_KEY_GOES_HERE
mo_census_data = c.acs5.get(("NAME", "B01003_001E", "B19301_001E"), {'for': 'county:*','in':'state:29'})
mo_census_pd = pd.DataFrame(mo_census_data)

# Column Reordering
mo_census_pd = mo_census_pd.rename(columns={"B01003_001E": "Total Population",
                                      "B19301_001E": "Per Capita Income",        
                                      "NAME": "County, State"})
                                   
mo_census_pd['FIPS']=mo_census_pd['state']+mo_census_pd['county']
mo_census_pd=mo_census_pd.drop(columns=['state','county'])
mo_census_pd=mo_census_pd.merge(covid_91120_data,how='left',on='FIPS').drop(columns={'County, State','Combined_Key'}).rename(columns={'Admin2':'County','Province_State':'State'})


# Final DataFrame
#census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       #"Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(mo_census_pd))
mo_census_pd.head()
mo_census_top = mo_census_pd.nlargest(10,["Total Population"])
#mo_census_top['County, State'].str.split(pat=',',expand=True)
#mo_census_top=mo_census_top.merge(mo_census_top['County, State'].str.split(pat=',',expand=True),how="outer", left_index=True, right_index=True) 
#mo_census_top = mo_census_top.rename(columns={0: "County",
#                                      1: "State"})
                                      #.drop(columns=["County, State"])
mo_census_top.head()

115


,Total Population,Per Capita Income,FIPS,County,State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
49,998684.0,39784.0,29189,St. Louis,Missouri,US,2020-09-12 04:25:05,38.637700,-90.445837,21343,763,0,20580.0,2146.740360,3.574943
65,692003.0,30237.0,29095,Jackson,Missouri,US,2020-09-12 04:25:05,39.010022,-94.347245,6313,85,0,6228.0,897.994484,1.346428
86,389985.0,37229.0,29183,St. Charles,Missouri,US,2020-09-12 04:25:05,38.778757,-90.666624,6468,105,0,6363.0,1608.867176,1.623377
91,311273.0,28478.0,29510,St. Louis City,Missouri,US,2020-09-12 04:25:05,38.635557,-90.243492,6527,192,0,6335.0,2171.497392,2.941627
36,288429.0,26378.0,29077,Greene,Missouri,US,2020-09-12 04:25:05,37.257587,-93.342309,4407,19,0,4388.0,1503.654218,0.431132


In [111]:
il_census_data = c.acs5.get(("NAME", "B01003_001E", "B19301_001E"), {'for': 'county:*','in':'state:17'})
il_census_pd = pd.DataFrame(il_census_data)
il_census_pd = il_census_pd.rename(columns={"B01003_001E": "Total Population",
                                      "B19301_001E": "Per Capita Income",        
                                      "NAME": "County, State"})

il_census_pd['FIPS']=il_census_pd['state']+il_census_pd['county']
il_census_pd=il_census_pd.drop(columns=['state','county'])
il_census_pd=il_census_pd.merge(covid_91120_data,how='left',on='FIPS').drop(columns={'County, State','Combined_Key'}).rename(columns={'Admin2':'County','Province_State':'State'})

print(len(il_census_pd))
il_census_pd.head()
il_census_top = il_census_pd.nlargest(10,["Total Population"])
#il_census_top['County, State'].str.split(pat=',',expand=True)
#il_census_top=il_census_top.merge(il_census_top['County, State'].str.split(pat=',',expand=True),how="outer", left_index=True, right_index=True) 
#il_census_top = il_census_top.rename(columns={0: "County",
#                                      1: "State"}).drop(columns=["County, State"])
il_census_top


102


,Total Population,Per Capita Income,FIPS,County,State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
38,5223719.0,35575.0,17031,Cook,Illinois,US,2020-09-12 04:25:05,41.841448,-87.816588,133686,5112,0,128574.0,2595.727222,3.823886
34,931743.0,43982.0,17043,DuPage,Illinois,US,2020-09-12 04:25:05,41.851170,-88.086427,15826,544,0,15282.0,1714.772987,3.437382
26,703619.0,44287.0,17097,Lake,Illinois,US,2020-09-12 04:25:05,42.322491,-88.003679,15583,477,0,15106.0,2237.217082,3.061028
18,688697.0,35259.0,17197,Will,Illinois,US,2020-09-12 04:25:05,41.446193,-87.978627,12681,367,0,12314.0,1835.849223,2.894094
54,530839.0,34924.0,17089,Kane,Illinois,US,2020-09-12 04:25:05,41.938427,-88.428614,11965,321,0,11644.0,2247.357735,2.682825
44,307789.0,38047.0,17111,McHenry,Illinois,US,2020-09-12 04:25:05,42.324512,-88.452482,4203,116,0,4087.0,1365.612430,2.759933
66,286174.0,28250.0,17201,Winnebago,Illinois,US,2020-09-12 04:25:05,42.336419,-89.160853,4731,148,0,4583.0,1674.263551,3.128303
86,265670.0,30802.0,17119,Madison,Illinois,US,2020-09-12 04:25:05,38.830425,-89.904226,4780,117,0,4663.0,1817.725485,2.447699
49,263463.0,29541.0,17163,St. Clair,Illinois,US,2020-09-12 04:25:05,38.469960,-89.930634,5924,178,0,5746.0,2281.216546,3.004727
97,209448.0,29683.0,17019,Champaign,Illinois,US,2020-09-12 04:25:05,40.139194,-88.200466,3876,20,0,3856.0,1848.451755,0.515996


In [112]:
id_census_data = c.acs5.get(("NAME", "B01003_001E", "B19301_001E"), {'for': 'county:*','in':'state:18'})
id_census_pd = pd.DataFrame(id_census_data)
id_census_pd = id_census_pd.rename(columns={"B01003_001E": "Total Population",
                                      "B19301_001E": "Per Capita Income",        
                                      "NAME": "County, State"})

id_census_pd['FIPS']=id_census_pd['state']+id_census_pd['county']
id_census_pd=id_census_pd.drop(columns=['state','county'])
id_census_pd=id_census_pd.merge(covid_91120_data,how='left',on='FIPS').drop(columns={'County, State','Combined_Key'}).rename(columns={'Admin2':'County','Province_State':'State'})

print(len(id_census_pd))
id_census_pd.head()
id_census_top = id_census_pd.nlargest(10,["Total Population"])
#id_census_top['County, State'].str.split(pat=',',expand=True)
#id_census_top=id_census_top.merge(id_census_top['County, State'].str.split(pat=',',expand=True),how="outer", left_index=True, right_index=True) 
#id_census_top = id_census_top.rename(columns={0: "County",
#                                      1: "State"}).drop(columns=["County, State"])
id_census_top

92


,Total Population,Per Capita Income,FIPS,County,State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
5,944523.0,27251.0,18097,Marion,Indiana,US,2020-09-12 04:25:05,39.781636,-86.138263,19549,754,0,18795.0,2026.680987,3.856975
61,486849.0,27660.0,18089,Lake,Indiana,US,2020-09-12 04:25:05,41.418743,-87.382776,9732,307,0,9425.0,2004.560313,3.154542
27,370016.0,28239.0,18003,Allen,Indiana,US,2020-09-12 04:25:05,41.091943,-85.068006,5519,189,0,5330.0,1455.052610,3.424533
54,316095.0,46597.0,18057,Hamilton,Indiana,US,2020-09-12 04:25:05,40.073065,-86.052447,4369,107,0,4262.0,1292.561485,2.449073
85,269240.0,27338.0,18141,St. Joseph,Indiana,US,2020-09-12 04:25:05,41.617568,-86.290408,5601,96,0,5505.0,2060.509296,1.713980
2,203604.0,25075.0,18039,Elkhart,Indiana,US,2020-09-12 04:25:05,41.597612,-85.858757,5937,103,0,5834.0,2877.275966,1.734883
51,189294.0,26262.0,18157,Tippecanoe,Indiana,US,2020-09-12 04:25:05,40.388727,-86.894140,1902,13,0,1889.0,971.736865,0.683491
37,181313.0,28320.0,18163,Vanderburgh,Indiana,US,2020-09-12 04:25:05,38.026588,-87.587631,2932,23,0,2909.0,1615.863236,0.784447
45,168041.0,32943.0,18127,Porter,Indiana,US,2020-09-12 04:25:05,41.459869,-87.068489,1886,43,0,1843.0,1106.878965,2.279958
7,160940.0,34689.0,18063,Hendricks,Indiana,US,2020-09-12 04:25:05,39.767190,-86.510373,2511,119,0,2392.0,1474.361609,4.739148


In [113]:
mi_census_data = c.acs5.get(("NAME", "B01003_001E", "B19301_001E"), {'for': 'county:*','in':'state:26'})
mi_census_pd = pd.DataFrame(mi_census_data)
mi_census_pd = mi_census_pd.rename(columns={"B01003_001E": "Total Population",
                                      "B19301_001E": "Per Capita Income",        
                                      "NAME": "County, State"})

mi_census_pd['FIPS']=mi_census_pd['state']+mi_census_pd['county']
mi_census_pd=mi_census_pd.drop(columns=['state','county'])
mi_census_pd=mi_census_pd.merge(covid_91120_data,how='left',on='FIPS').drop(columns={'County, State','Combined_Key'}).rename(columns={'Admin2':'County','Province_State':'State'})

print(len(mi_census_pd))
mi_census_pd.head()
mi_census_top = mi_census_pd.nlargest(10,["Total Population"])
#mi_census_top['County, State'].str.split(pat=',',expand=True)
#mi_census_top=mi_census_top.merge(mi_census_top['County, State'].str.split(pat=',',expand=True),how="outer", left_index=True, right_index=True) 
#mi_census_top = mi_census_top.rename(columns={0: "County",
#                                      1: "State"}).drop(columns=["County, State"])
mi_census_top

83


,Total Population,Per Capita Income,FIPS,County,State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
30,1761382.0,25976.0,26163,Wayne,Michigan,US,2020-09-12 04:25:05,42.280984,-83.281255,33113,2940,0,30173.0,1892.882071,8.878688
61,1250843.0,42760.0,26125,Oakland,Michigan,US,2020-09-12 04:25:05,42.660901,-83.385954,19272,1183,0,18089.0,1532.462245,6.138439
50,868704.0,31104.0,26099,Macomb,Michigan,US,2020-09-12 04:25:05,42.691584,-82.927528,13599,1001,0,12598.0,1555.999506,7.360835
49,643140.0,31005.0,26081,Kent,Michigan,US,2020-09-12 04:25:05,43.031977,-85.549346,8963,171,0,8792.0,1364.324802,1.907843
79,409361.0,26386.0,26049,Genesee,Michigan,US,2020-09-12 04:25:05,43.022442,-83.705897,4096,305,0,3791.0,1009.331884,7.446289
25,365961.0,39486.0,26161,Washtenaw,Michigan,US,2020-09-12 04:25:05,42.253105,-83.838532,3127,114,0,3013.0,850.650570,3.645667
68,289564.0,28366.0,26065,Ingham,Michigan,US,2020-09-12 04:25:05,42.597169,-84.374721,2424,42,0,2382.0,828.984357,1.732673
9,284034.0,30558.0,26139,Ottawa,Michigan,US,2020-09-12 04:25:05,42.959968,-85.996911,2830,64,0,2766.0,969.742658,2.261484
40,261573.0,30555.0,26077,Kalamazoo,Michigan,US,2020-09-12 04:25:05,42.245422,-85.530386,2239,91,0,2148.0,844.695283,4.064314
52,192778.0,26625.0,26145,Saginaw,Michigan,US,2020-09-12 04:25:05,43.334339,-84.051312,2743,133,0,2610.0,1439.600292,4.848706


In [114]:
oh_census_data = c.acs5.get(("NAME", "B01003_001E", "B19301_001E"), {'for': 'county:*','in':'state:39'})
oh_census_pd = pd.DataFrame(oh_census_data)
oh_census_pd = oh_census_pd.rename(columns={"B01003_001E": "Total Population",
                                      "B19301_001E": "Per Capita Income",        
                                      "NAME": "County, State"})

oh_census_pd['FIPS']=oh_census_pd['state']+oh_census_pd['county']
oh_census_pd=oh_census_pd.drop(columns=['state','county'])
oh_census_pd=oh_census_pd.merge(covid_91120_data,how='left',on='FIPS').drop(columns={'County, State','Combined_Key'}).rename(columns={'Admin2':'County','Province_State':'State'})

print(len(oh_census_pd))
oh_census_pd.head()
oh_census_top = oh_census_pd.nlargest(10,["Total Population"])
#oh_census_top['County, State'].str.split(pat=',',expand=True)
#oh_census_top=oh_census_top.merge(oh_census_top['County, State'].str.split(pat=',',expand=True),how="outer", left_index=True, right_index=True) 
#oh_census_top = oh_census_top.rename(columns={0: "County",
#                                      1: "State"}).drop(columns=["County, State"])
oh_census_top

88


,Total Population,Per Capita Income,FIPS,County,State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
9,1275333.0,32552.0,39049,Franklin,Ohio,US,2020-09-12 04:25:05,39.969958,-83.011158,24144,577,0,23567.0,1833.597113,2.389828
56,1253783.0,31804.0,39035,Cuyahoga,Ohio,US,2020-09-12 04:25:05,41.424119,-81.659183,16544,621,0,15923.0,1339.517048,3.753627
76,812037.0,34125.0,39061,Hamilton,Ohio,US,2020-09-12 04:25:05,39.196736,-84.545029,12067,291,0,11776.0,1476.134380,2.411536
29,541810.0,32055.0,39153,Summit,Ohio,US,2020-09-12 04:25:05,41.124647,-81.531231,4814,244,0,4570.0,889.812260,5.068550
59,532034.0,28807.0,39113,Montgomery,Ohio,US,2020-09-12 04:25:05,39.753949,-84.290510,6961,143,0,6818.0,1309.228926,2.054303
42,432379.0,28361.0,39095,Lucas,Ohio,US,2020-09-12 04:25:05,41.621012,-83.654686,6810,350,0,6460.0,1589.828831,5.139501
14,378294.0,30777.0,39017,Butler,Ohio,US,2020-09-12 04:25:05,39.440128,-84.573887,5124,87,0,5037.0,1337.391096,1.697892
47,373475.0,28434.0,39151,Stark,Ohio,US,2020-09-12 04:25:05,40.814825,-81.364373,2528,158,0,2370.0,682.126031,6.250000
57,306713.0,29777.0,39093,Lorain,Ohio,US,2020-09-12 04:25:05,41.295538,-82.150835,2197,85,0,2112.0,709.091672,3.868912
72,231064.0,27429.0,39099,Mahoning,Ohio,US,2020-09-12 04:25:05,41.016311,-80.772870,2935,274,0,2661.0,1283.436023,9.335605
